# This notebook fine-tunes the Vidgen et al. (2021) model using the entire fine-tuning dataset without cross-validation.

Importing required libraries and modules.

In [ ]:
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import torch

In [ ]:
!pip install transformers

Loading data.

In [ ]:
data = pd.read_csv('Notebook_8_9_10_fine_tune_final.csv')

Importing required libraries and modules and model

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments

# 1. Load the pre-trained model and tokenizer
model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Logging into Huggingface to allow for fine-tune model to be uploaded onto account.

In [ ]:
!pip install huggingface_hub
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

Processing data.

In [ ]:
!huggingface-cli whoami

EZiisk


Importing required libraries and modules and preprocessing sentences.

In [ ]:
import re
import string

def preprocess_sentence(sentence):
  # no lowercasing or punctuation removal as assumed to carry semantic information
    sentence = re.sub(r'\\n', ' ', sentence)
    sentence = re.sub(r'\s+', ' ', sentence).strip()
    return sentence

# Apply the preprocess_sentence function to the 'sentences' column
data['sentences'] = data['sentences'].apply(preprocess_sentence)


Creating the train/validation/test split by stratifying the data using the gold label column.

In [ ]:
# Group data by "gold_label" and create lists of texts and labels for each group
grouped_data = data.groupby('gold_label')
grouped_texts = [group['sentences'].tolist() for _, group in grouped_data]
grouped_labels = [group['hate_label'].tolist() for _, group in grouped_data]

# Initialize lists to store train, validation, and test data
train_texts, val_texts, test_texts = [], [], []
train_labels, val_labels, test_labels = [], [], []

# Split each group into train, validation, and test sets
for texts, labels in zip(grouped_texts, grouped_labels):
    train_texts_group, test_texts_group, train_labels_group, test_labels_group = train_test_split(texts, labels, test_size=0.15, stratify=labels, random_state=42)
    train_texts_group, val_texts_group, train_labels_group, val_labels_group = train_test_split(train_texts_group, train_labels_group, test_size=0.1765, stratify=train_labels_group, random_state=42)

    train_texts.extend(train_texts_group)
    val_texts.extend(val_texts_group)
    test_texts.extend(test_texts_group)
    train_labels.extend(train_labels_group)
    val_labels.extend(val_labels_group)
    test_labels.extend(test_labels_group)


Creating the custom DataLoader.

In [ ]:
# Define a custom dataset class for hate speech detection using PyTorch
class HateSpeechDataset(torch.utils.data.Dataset):

    # Initialize the dataset object
    def __init__(self, texts, labels, tokenizer, max_len):
        # Store the list of textual samples
        self.texts = texts
        # Store the list of labels corresponding to each text sample
        self.labels = labels
        # Store the tokenizer instance which will convert text to tokens
        self.tokenizer = tokenizer
        # Store the maximum token length for sequences
        self.max_len = max_len

    # Return the total number of samples in the dataset
    def __len__(self):
        return len(self.texts)

    # Fetch and return a single data sample given its index
    def __getitem__(self, item):
        # Retrieve the text and its corresponding label using the provided index
        text = self.texts[item]
        label = self.labels[item]

        # Tokenize the text using the provided tokenizer
        # This converts the text to a format suitable for model input
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,   # Add special tokens like [CLS], [SEP]
            max_length=self.max_len,   # Ensure the sequence doesn't exceed the max length
            padding='max_length',      # Pad short sequences to the max length
            truncation=True,           # Truncate sequences exceeding the max length
            return_tensors='pt'        # Return data as PyTorch tensors
        )

        # Return a dictionary containing the tokenized data and the label
        return {
            # The token IDs of the text
            'input_ids': encoding['input_ids'].flatten(),
            # A mask to indicate real tokens (1) vs padded tokens (0)
            'attention_mask': encoding['attention_mask'].flatten(),
            # The corresponding label of the text sample
            'labels': torch.tensor(label, dtype=torch.long)
        }


Creating the train/validation and test datasets

In [ ]:
# Step 4: Create Datasets
max_len = 128
train_dataset = HateSpeechDataset(train_texts, train_labels, tokenizer, max_len)
val_dataset = HateSpeechDataset(val_texts, val_labels, tokenizer, max_len)
test_dataset = HateSpeechDataset(test_texts, test_labels, tokenizer, max_len)

In [ ]:
!pip install accelerate -U transformers[torch]

Importing required libraries and modules, and running the training and evaluation loops using the Hugginface Trainer class.

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
from transformers import EvalPrediction# Define evaluation metrics function
import numpy as np

def compute_metrics(p: EvalPrediction):
    preds = np.argmax(p.predictions, axis=1)
    return {
        'accuracy': accuracy_score(p.label_ids, preds),
        'f1': f1_score(p.label_ids, preds, average='weighted')
    }

# Step 5: Fine-tuning the Model using the Trainer class
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=500,
    evaluation_strategy='steps',
    eval_steps=500,
    load_best_model_at_end=True,  # Set load_best_model_at_end to True
    # Remove 'early_stopping_patience' from here
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],  # Use EarlyStoppingCallback here
)

# Train the model
trainer.train()

# Step 6: Evaluate on Test Set
eval_result = trainer.evaluate(test_dataset)
print(f"Test Accuracy: {eval_result['eval_accuracy']:.4f}, Test F1: {eval_result['eval_f1']:.4f}")

# Get the predicted labels for the test dataset
test_predictions = trainer.predict(test_dataset).predictions
test_predicted_labels = np.argmax(test_predictions, axis=1)

# Create a dictionary to store the collected information
test_data_dict = {
    "original_sentence": test_texts,  # Original sentences from the test dataset
    "hate_label": test_labels,  # Ground truth hate labels from the HateSpeechDataset
    "predicted_label": test_predicted_labels,  # Predicted labels from the model
}

# Create a DataFrame from the dictionary
test_results_df = pd.DataFrame(test_data_dict)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Step,Training Loss,Validation Loss,Accuracy,F1
500,0.454300,0.402625,0.849250,0.845649
1000,0.335900,0.318017,0.855959,0.856966
1500,0.278400,0.333650,0.883189,0.883659
2000,0.189400,0.387020,0.887135,0.886834


Test Accuracy: 0.8469, Test F1: 0.8480


Producing the dataframe of the test sentences, their ground truth labels and the predicted labels from the model.

In [ ]:
# Inner merge based on the condition where 'sentences' matches 'original_sentence'
merged_df = data.merge(test_results_df, left_on='sentences', right_on='original_sentence', how='inner')

# Drop the duplicate 'original_sentence' column after the merge
merged_df.drop('original_sentence', axis=1, inplace=True)

In [ ]:
columns_to_drop = ['hate_label_y']
merged_df = merged_df.drop(columns_to_drop, axis=1)

In [ ]:
column_name_mapping = {
    'hate_label_x': 'hate_label',
}

# Rename the columns using the dictionary
merged_df.rename(columns=column_name_mapping, inplace=True)

In [ ]:
columns_to_drop = ['clean_sentences']
merged_df = merged_df.drop(columns_to_drop, axis=1)

Downloading the CSV file

In [ ]:
from google.colab import files

merged_df.to_csv("2.55_finetune_test_dataset_analysis", index=False)
files.download("2.55_finetune_test_dataset_analysis")

Saving the model to the Huggingface Hub

In [ ]:
# model_path = "EZiisk/EZ_finetune_Vidgen_model_RHS"
# save_path = "EZiisk/EZ_finetune_Vidgen_model_RHS_tokenizer"
# model.save_pretrained(model_path, push_to_hub = True)
# tokenizer.save_pretrained(save_path, push_to_hub = True)